# Capstone Project - The Battle of Neighborhoods Week 2 Notebook

In [1]:
# let's download all the required dependencies
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analysis 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from pandas.io.json import json_normalize # Utility function json_normalize for flattening semi-structured JSON objects
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
wikiPostalCodes = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(wikiPostalCodes,'lxml') 

In [4]:
result = soup.prettify().splitlines()
print('\n'.join(result[:20] + result[-20:]))

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":890001695,"wgRevisionId":890001695,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

In [5]:
My_table = soup.find('table',{'class':'wikitable sortable'})
My_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [6]:
rows = My_table.findAll('tr')
rows

[<tr>
 <th>Postcode</th>
 <th>Borough</th>
 <th>Neighbourhood
 </th></tr>, <tr>
 <td>M1A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M2A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M3A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td></tr>, <tr>
 <td>M4A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td></tr>, <tr>
 <td>M5A</td>
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
 <td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
 </td></tr>, <tr>
 <td>M5A</td>
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
 <td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
 </td></tr>, <tr>
 <td>M6A</td>
 <td

In [7]:
parsed_data = []

In [8]:
for row in rows:
    children = row.findChildren(recursive=False)
    row_text = []
    for child in children: 
        clean_text = child.text 
        clean_text = clean_text.split('&#91;')[0] # This is to discard reference/citation links
        clean_text = clean_text.split('&#160;')[-1] # This is to clean the header row of the sort icons
        clean_text = clean_text.strip()
        row_text.append(clean_text)
    parsed_data.append(row_text)

In [9]:
parsed_data[:5]

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village']]

In [10]:
# Define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood']

# Instantiate and populate the dataframe
df = pd.DataFrame(parsed_data[1:], columns=column_names)

# Examine the resulting dataframe
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [11]:
# Process the cells that have an assigned borough. Ignore cells with a borough that is not assigned.
df.drop(df[df['Borough']=='Not assigned'].index, inplace=True)
df.reset_index(inplace=True, drop=True)
print("The new number of rows in dataframe after dropping unassigned boroughs:", df.shape[0])

The new number of rows in dataframe after dropping unassigned boroughs: 211


In [12]:
# The neighborhood will be the same as the borough if a cell has a borough but a Not assigned neighborhood.
df['Neighborhood'].where(df['Neighborhood'] != 'Not assigned', df['Borough'], inplace=True)

In [13]:
# More than one neighborhood can exist in one postal code area. 
# Combined the rows into one row with the neighborhoods separated with a comma
df=df.groupby("PostalCode").agg(lambda x:','.join(set(x)))

In [14]:
df=df.reset_index()

In [15]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
print("The number of rows in dataframe:", df.shape[0])
df.shape

The number of rows in dataframe: 103


(103, 3)

In [17]:
df_copy = df.copy() # Make a copy of the dataframe

In [18]:
geodata = pd.read_csv('https://cocl.us/Geospatial_data')
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
geodata.rename(index=str, columns={"Postal Code":"PostalCode"},inplace=True)
geodata.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
# Merge the original df_copy with geodata
df = df_copy.merge(geodata, how='inner', on='PostalCode')

In [21]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [22]:
# Check how many boroughs and neighborhoods there are
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [23]:
import folium # Import Folium visualization library

In [24]:
# Segment and Cluster by Downtown Toronto
tor_data = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
tor_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town,Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [25]:
tor_data.tail()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
13,M5T,Downtown Toronto,"Kensington Market,Grange Park,Chinatown",43.653206,-79.400049
14,M5V,Downtown Toronto,"South Niagara,Bathurst Quay,King and Spadina,R...",43.628947,-79.394420
15,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846
16,M5X,Downtown Toronto,"Underground city,First Canadian Place",43.648429,-79.382280
17,M6G,Downtown Toronto,Christie,43.669542,-79.422564


In [26]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [27]:
# Use geopy library to get the latitude and longitude values of Toronto.
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [29]:
# Create a map of Downtown Toronto using Latitude and Longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=13)

# Add markers to map
for lat, lng, label in zip(tor_data['Latitude'], tor_data['Longitude'], tor_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

In [30]:
# Next, I am going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
CLIENT_ID = 'RDZDO5MSITL4N20HVKR2WPZ1RHFP3JVEI1OZHZLTRJC1MYMX' # Foursquare ID
CLIENT_SECRET = 'WB3GMUZTGTJCTXN0CSCJGG54F0UBXRE1IWC5WMDJ1KSCAM4X' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentails:
CLIENT_ID: RDZDO5MSITL4N20HVKR2WPZ1RHFP3JVEI1OZHZLTRJC1MYMX
CLIENT_SECRET:WB3GMUZTGTJCTXN0CSCJGG54F0UBXRE1IWC5WMDJ1KSCAM4X


In [31]:
tor_data.loc[0, 'Neighborhood'] # Get the neighborhood name

'Rosedale'

In [32]:
# Get the neighborhoods' latitude and longitude values
neighborhood_latitude = tor_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = tor_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = tor_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


In [33]:
# Let's get the top 100 venues that are in Rosedale within a radius of 500 meters.
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id=RDZDO5MSITL4N20HVKR2WPZ1RHFP3JVEI1OZHZLTRJC1MYMX&client_secret=WB3GMUZTGTJCTXN0CSCJGG54F0UBXRE1IWC5WMDJ1KSCAM4X&v=20180605&ll=43.6056466,-79.50132070000001&radius=500&limit=100'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=RDZDO5MSITL4N20HVKR2WPZ1RHFP3JVEI1OZHZLTRJC1MYMX&client_secret=WB3GMUZTGTJCTXN0CSCJGG54F0UBXRE1IWC5WMDJ1KSCAM4X&v=20180605&ll=43.6056466,-79.50132070000001&radius=500&limit=100'

In [34]:
# Send the GET request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cd0ea1b4c1f6743c883ccfc'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 15,
  'suggestedBounds': {'ne': {'lat': 43.6101466045, 'lng': -79.49511771930959},
   'sw': {'lat': 43.6011465955, 'lng': -79.50752368069043}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b119977f964a520488023e3',
       'name': 'LCBO',
       'location': {'address': '2762 Lake Shore Blvd W',
        'crossStreet': 'btwn 1st & 2nd St',
        'lat': 43.60228082768786,
        'lng': -79.4993016827402,
        'labeledLatLngs': [{'label': 'display',
          'lat':

In [35]:
# All the information is in the items key. Let's borrow the get_category_type function from the Foursquare lab.
# Function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [36]:
# Clean the json and structure in into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,LCBO,Liquor Store,43.602281,-79.499302
1,Domino's Pizza,Pizza Place,43.601676,-79.500908
2,New Toronto Fish & Chips,Restaurant,43.601849,-79.503281
3,Delicia Bakery & Pastry,Bakery,43.601403,-79.503012
4,Lucky Dice Restaurant,Café,43.601392,-79.503056


In [37]:
# How many values were returned by Foursquare?
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

15 venues were returned by Foursquare.


In [38]:
# Use the function from the lab to repeat the same process to all the neighborhoods in Downtown Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
# Run the above function on each neighborhood and create a new dataframe called tor_venues.
tor_venues = getNearbyVenues(names=tor_data['Neighborhood'],
                                   latitudes=tor_data['Latitude'],
                                   longitudes=tor_data['Longitude']
                                  )

Rosedale
St. James Town,Cabbagetown
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,Richmond,King
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
University of Toronto,Harbord
Kensington Market,Grange Park,Chinatown
South Niagara,Bathurst Quay,King and Spadina,Railway Lands,CN Tower,Harbourfront West,Island airport
Stn A PO Boxes 25 The Esplanade
Underground city,First Canadian Place
Christie


In [40]:
# Check the size of the resulting dataframe
print(tor_venues.shape)
tor_venues.head()

(1287, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town,Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


In [41]:
tor_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1282,Christie,43.669542,-79.422564,Queens Club,43.672386,-79.418106,Athletics & Sports
1283,Christie,43.669542,-79.422564,Pioneer Gas,43.670355,-79.428400,Convenience Store
1284,Christie,43.669542,-79.422564,Marian Engel Park,43.673754,-79.423988,Park
1285,Christie,43.669542,-79.422564,Garrison Creek Park,43.671690,-79.427805,Park
1286,Christie,43.669542,-79.422564,Foto Grocery,43.667979,-79.428217,Grocery Store


In [42]:
# Find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

There are 204 uniques categories.


In [43]:
# tor_venues['Venue Category'] = 'Restaurant' 
tor_venuestest = tor_venues[tor_venues['Venue Category'].str.contains('staurant')]

In [44]:
# Find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

There are 204 uniques categories.


In [45]:
# Analyze each neighborhood
# one hot encoding
tor_onehot = pd.get_dummies(tor_venuestest[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_venuestest['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Dim Sum Restaurant,Doner Restaurant,Dumpling Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,New American Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,"Adelaide,Richmond,King",0.000000,0.148148,0.074074,0.000000,0.037037,0.000000,0.000000,0.037037,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.037037,0.037037,0.037037,0.037037,0.074074,0.000000,0.037037,0.037037,0.000000,0.000000,0.000000,0.0,0.037037,0.000000,0.000000,0.074074,0.037037,0.074074,0.0,0.148148,0.000000,0.037037,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.090909,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.090909,0.0,0.000000,0.090909,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.181818,0.181818,0.000000,0.0,0.090909,0.000000,0.090909,0.000000
2,Central Bay Street,0.000000,0.035714,0.000000,0.000000,0.000000,0.035714,0.071429,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.035714,0.000000,0.0,0.035714,0.0,0.000000,0.000000,0.071429,0.142857,0.071429,0.035714,0.000000,0.035714,0.000000,0.071429,0.035714,0.0,0.000000,0.035714,0.035714,0.071429,0.035714,0.071429,0.0,0.035714,0.000000,0.035714,0.000000
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.034483,0.034483,0.000000,0.000000,0.000000,0.034483,0.034483,0.000000,0.000000,0.0,0.0,0.0,0.034483,0.000000,0.068966,0.0,0.000000,0.0,0.000000,0.000000,0.034483,0.034483,0.206897,0.000000,0.000000,0.068966,0.034483,0.000000,0.000000,0.0,0.000000,0.000000,0.034483,0.103448,0.034483,0.137931,0.0,0.000000,0.034483,0.000000,0.034483


In [46]:
tor_venuestest.head(20)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5,"St. James Town,Cabbagetown",43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
6,"St. James Town,Cabbagetown",43.667967,-79.367675,F'Amelia,43.667536,-79.368613,Italian Restaurant
7,"St. James Town,Cabbagetown",43.667967,-79.367675,Kingyo Toronto,43.665895,-79.368415,Japanese Restaurant
9,"St. James Town,Cabbagetown",43.667967,-79.367675,Murgatroid,43.667381,-79.369311,Restaurant
18,"St. James Town,Cabbagetown",43.667967,-79.367675,Kanpai Snack Bar,43.664331,-79.368065,Taiwanese Restaurant
19,"St. James Town,Cabbagetown",43.667967,-79.367675,Mr. Jerk,43.667328,-79.373389,Caribbean Restaurant
24,"St. James Town,Cabbagetown",43.667967,-79.367675,Thai Room - Cabbagetown,43.664159,-79.368189,Thai Restaurant
25,"St. James Town,Cabbagetown",43.667967,-79.367675,The Pear Tree,43.664904,-79.368246,Restaurant
27,"St. James Town,Cabbagetown",43.667967,-79.367675,Hey Lucy,43.664075,-79.368655,Italian Restaurant
41,"St. James Town,Cabbagetown",43.667967,-79.367675,Cabbage Town Kitchen,43.668181,-79.369570,American Restaurant


In [47]:
tor_grouped.tail()

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Dim Sum Restaurant,Doner Restaurant,Dumpling Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,New American Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
11,St. James Town,0.0,0.080000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.040000,0.04,0.00,0.00,0.040000,0.120000,0.080000,0.04,0.04,0.00,0.0,0.04,0.0,0.04,0.04,0.0,0.0,0.200000,0.08,0.000000,0.000000,0.040000,0.0,0.04,0.0
12,"St. James Town,Cabbagetown",0.0,0.090909,0.00,0.00,0.00,0.090909,0.090909,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.00,0.00,0.00,0.090909,0.181818,0.090909,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.181818,0.00,0.000000,0.090909,0.090909,0.0,0.00,0.0
13,Stn A PO Boxes 25 The Esplanade,0.0,0.050000,0.00,0.05,0.00,0.000000,0.000000,0.00,0.050000,0.0,0.0,0.0,0.0,0.0,0.10,0.0,0.050000,0.00,0.00,0.00,0.000000,0.100000,0.100000,0.00,0.00,0.00,0.0,0.00,0.0,0.05,0.00,0.0,0.0,0.200000,0.15,0.000000,0.000000,0.050000,0.0,0.05,0.0
14,"Underground city,First Canadian Place",0.0,0.120000,0.08,0.00,0.04,0.000000,0.000000,0.04,0.000000,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.000000,0.00,0.04,0.04,0.000000,0.040000,0.080000,0.00,0.00,0.04,0.0,0.00,0.0,0.00,0.04,0.0,0.0,0.120000,0.12,0.040000,0.000000,0.080000,0.0,0.04,0.0
15,"University of Toronto,Harbord",0.0,0.000000,0.00,0.00,0.00,0.000000,0.111111,0.00,0.111111,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.111111,0.00,0.00,0.00,0.000000,0.111111,0.222222,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.222222,0.00,0.111111,0.000000,0.000000,0.0,0.00,0.0


In [48]:
tor_grouped.shape

(16, 42)

In [49]:
# Examine the new dataframe size
tor_onehot.shape

(311, 42)

In [50]:
# Let's confirm the new size
tor_grouped.shape

(16, 42)

In [51]:
tor_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Dim Sum Restaurant,Doner Restaurant,Dumpling Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,New American Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,"Adelaide,Richmond,King",0.000000,0.148148,0.074074,0.000000,0.037037,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.037037,0.037037,0.037037,0.037037,0.074074,0.000000,0.037037,0.037037,0.000000,0.000000,0.000000,0.00,0.037037,0.000000,0.000000,0.074074,0.037037,0.074074,0.000000,0.148148,0.000000,0.037037,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.00,0.000000,0.090909,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.181818,0.181818,0.000000,0.000000,0.090909,0.000000,0.090909,0.000000
2,Central Bay Street,0.000000,0.035714,0.000000,0.000000,0.000000,0.035714,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.035714,0.00,0.000000,0.000000,0.071429,0.142857,0.071429,0.035714,0.000000,0.035714,0.000000,0.071429,0.035714,0.00,0.000000,0.035714,0.035714,0.071429,0.035714,0.071429,0.000000,0.035714,0.000000,0.035714,0.000000
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.034483,0.034483,0.000000,0.000000,0.000000,0.034483,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.068966,0.000000,0.000000,0.00,0.000000,0.000000,0.034483,0.034483,0.206897,0.000000,0.000000,0.068966,0.034483,0.000000,0.000000,0.00,0.000000,0.000000,0.034483,0.103448,0.034483,0.137931,0.000000,0.000000,0.034483,0.000000,0.034483
5,"Commerce Court,Victoria Hotel",0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.038462,0.00,0.038462,0.038462,0.000000,0.115385,0.076923,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.00,0.038462,0.000000,0.000000,0.192308,0.115385,0.000000,0.000000,0.076923,0.000000,0.038462,0.000000
6,"Design Exchange,Toronto Dominion Centre",0.000000,0.130435,0.043478,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.043478,0.00,0.043478,0.043478,0.000000,0.130435,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.043478,0.000000,0.000000,0.217391,0.130435,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000
7,"Harbourfront East,Toronto Islands,Union Station",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.00,0.000000,0.000000,0.066667,0.266667,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.066667,0.000000,0.000000,0.200000,0.066667,0.066667,0.000000,0.000000,0.000000,0.066667,0.000000
8,"Harbourfront,Regent Park",0.00

In [52]:
# Let's print each neighborhood along with the top 10 most common venues
num_top_venues = 10

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,Richmond,King----
                      venue  freq
0       American Restaurant  0.15
1           Thai Restaurant  0.15
2          Asian Restaurant  0.07
3       Japanese Restaurant  0.07
4          Sushi Restaurant  0.07
5                Restaurant  0.07
6         Indian Restaurant  0.04
7  Mediterranean Restaurant  0.04
8        Italian Restaurant  0.04
9          Greek Restaurant  0.04


----Berczy Park----
                           venue  freq
0             Seafood Restaurant  0.18
1                     Restaurant  0.18
2  Vegetarian / Vegan Restaurant  0.09
3             Belgian Restaurant  0.09
4             Italian Restaurant  0.09
5                Thai Restaurant  0.09
6               Greek Restaurant  0.09
7        Comfort Food Restaurant  0.09
8              French Restaurant  0.09
9              Afghan Restaurant  0.00


----Central Bay Street----
                        venue  freq
0          Italian Restaurant  0.14
1           Indian Restaurant  0.07
2      

In [54]:
temp # inspect temp

,venue,freq
1,Afghan Restaurant,0.00
2,American Restaurant,0.00
3,Asian Restaurant,0.00
4,Belgian Restaurant,0.00
5,Brazilian Restaurant,0.00
6,Caribbean Restaurant,0.00
7,Chinese Restaurant,0.11
8,Colombian Restaurant,0.00
9,Comfort Food Restaurant,0.11
10,Dim Sum Restaurant,0.00


In [55]:
# Let's put that into a pandas dataframe
# Use the function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [56]:
# Create the new dataframe and display the top 15 venues for each neighborhood
num_top_venues = 15
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(16)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,"Adelaide,Richmond,King",American Restaurant,Thai Restaurant,Asian Restaurant,Sushi Restaurant,Restaurant,Japanese Restaurant,Indian Restaurant,Mediterranean Restaurant,Gluten-free Restaurant,Greek Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Latin American Restaurant,Colombian Restaurant,New American Restaurant
1,Berczy Park,Seafood Restaurant,Restaurant,Greek Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Belgian Restaurant,Italian Restaurant,French Restaurant,Comfort Food Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Colombian Restaurant
2,Central Bay Street,Italian Restaurant,Indian Restaurant,Sushi Restaurant,Japanese Restaurant,Restaurant,Chinese Restaurant,Middle Eastern Restaurant,Modern European Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Mediterranean Restaurant,Caribbean Restaurant,Portuguese Restaurant,Ramen Restaurant
3,Christie,Italian Restaurant,Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Doner Restaurant,Comfort Food Restaurant,Gluten-free Restaurant,Colombian Restaurant,Chinese Restaurant
4,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Restaurant,Fast Food Restaurant,Mediterranean Restaurant,Vietnamese Restaurant,Mexican Restaurant,American Restaurant,Caribbean Restaurant,Chinese Restaurant,Ethiopian Restaurant,Italian Restaurant,Indian Restaurant,Afghan Restaurant,Seafood Restaurant
5,"Commerce Court,Victoria Hotel",Restaurant,American Restaurant,Seafood Restaurant,Italian Restaurant,Thai Restaurant,Japanese Restaurant,Latin American Restaurant,French Restaurant,Gluten-free Restaurant,Greek Restaurant,New American Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Ethiopian Restaurant
6,"Design Exchange,Toronto Dominion Centre",Restaurant,American Restaurant,Seafood Restaurant,Italian Restaurant,French Restaurant,Asian Restaurant,Fast Food Restaurant,Gluten-free Restaurant,Greek Restaurant,Chinese Restaurant,New American Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Ethiopian Restaurant,Falafel Restaurant
7,"Harbourfront East,Toronto Islands,Union Station",Italian Restaurant,Restaurant,Indian Restaurant,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Seafood Restaurant,Japanese Restaurant,New American Restaurant,Chinese Restaurant,Doner Restaurant,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Colombian Restaurant
8,"Harbourfront,Regent Park",Mexican Restaurant,French Restaurant,Italian Restaurant,Restaurant,Vietnamese Restaurant,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,Comfort Food Restaurant,Colombian Restaurant
9,"Kensington Market,Grange Park,Chinatown",Vegetarian / Vegan Restaurant,Mexican Restaurant,Dumpling Restaurant,Vietnamese Restaurant,Chinese Restaurant,Caribbean Restaurant,Dim Sum Restaurant,Ramen Restaurant,Comfort Food Restaurant,Doner Restaurant,Belgian Restaurant,Japanese Restaurant,Italian Restaurant,Filipino Restaurant,Brazilian Restaurant


In [57]:
# Run k-means to cluster the neighborhood into 5 clusters
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 1, 0, 2, 2, 2, 3, 4], dtype=int32)

In [58]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = tor_data

# merge to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged.head() # check the columns

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M4X,Downtown Toronto,"St. James Town,Cabbagetown",43.667967,-79.367675,2.0,Restaurant,Italian Restaurant,Indian Restaurant,American Restaurant,Thai Restaurant,Taiwanese Restaurant,Caribbean Restaurant,Chinese Restaurant,Japanese Restaurant,Dumpling Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Colombian Restaurant
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0.0,Japanese Restaurant,Sushi Restaurant,Restaurant,Fast Food Restaurant,Mediterranean Restaurant,Vietnamese Restaurant,Mexican Restaurant,American Restaurant,Caribbean Restaurant,Chinese Restaurant,Ethiopian Restaurant,Italian Restaurant,Indian Restaurant,Afghan Restaurant,Seafood Restaurant
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,3.0,Mexican Restaurant,French Restaurant,Italian Restaurant,Restaurant,Vietnamese Restaurant,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,Comfort Food Restaurant,Colombian Restaurant
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0.0,Middle Eastern Restaurant,Italian Restaurant,Japanese Restaurant,Restaurant,Ramen Restaurant,Fast Food Restaurant,Modern European Restaurant,Ethiopian Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Mexican Restaurant,Vietnamese Restaurant,Caribbean Restaurant,Seafood Restaurant,Sushi Restaurant


In [59]:
# Ignore/drop NaNs
tor_merged.dropna(axis=0, how='any',inplace=True)
tor_merged.reset_index(inplace=True, drop=True)
print("Number of rows after dropping NaNs:", len(tor_merged))
print("Number of NaNs:", tor_merged.isna().sum())

Number of rows after dropping NaNs: 16
Number of NaNs: PostalCode                0
Borough                   0
Neighborhood              0
Latitude                  0
Longitude                 0
Cluster Labels            0
1st Most Common Venue     0
2nd Most Common Venue     0
3rd Most Common Venue     0
4th Most Common Venue     0
5th Most Common Venue     0
6th Most Common Venue     0
7th Most Common Venue     0
8th Most Common Venue     0
9th Most Common Venue     0
10th Most Common Venue    0
11th Most Common Venue    0
12th Most Common Venue    0
13th Most Common Venue    0
14th Most Common Venue    0
15th Most Common Venue    0
dtype: int64


In [60]:
# Visualize the Clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [61]:
tor_merged.groupby('Cluster Labels').count()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,,,,,,
0.0,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
1.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2.0,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
3.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [62]:
# Cluster 1
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[2] + list(range(6, tor_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
1,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Restaurant,Fast Food Restaurant,Mediterranean Restaurant,Vietnamese Restaurant,Mexican Restaurant,American Restaurant,Caribbean Restaurant,Chinese Restaurant,Ethiopian Restaurant,Italian Restaurant,Indian Restaurant,Afghan Restaurant,Seafood Restaurant
3,"Ryerson,Garden District",Middle Eastern Restaurant,Italian Restaurant,Japanese Restaurant,Restaurant,Ramen Restaurant,Fast Food Restaurant,Modern European Restaurant,Ethiopian Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Mexican Restaurant,Vietnamese Restaurant,Caribbean Restaurant,Seafood Restaurant,Sushi Restaurant
6,Central Bay Street,Italian Restaurant,Indian Restaurant,Sushi Restaurant,Japanese Restaurant,Restaurant,Chinese Restaurant,Middle Eastern Restaurant,Modern European Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Mediterranean Restaurant,Caribbean Restaurant,Portuguese Restaurant,Ramen Restaurant
7,"Adelaide,Richmond,King",American Restaurant,Thai Restaurant,Asian Restaurant,Sushi Restaurant,Restaurant,Japanese Restaurant,Indian Restaurant,Mediterranean Restaurant,Gluten-free Restaurant,Greek Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Latin American Restaurant,Colombian Restaurant,New American Restaurant
14,"Underground city,First Canadian Place",American Restaurant,Seafood Restaurant,Restaurant,Thai Restaurant,Asian Restaurant,Japanese Restaurant,Colombian Restaurant,Gluten-free Restaurant,Vegetarian / Vegan Restaurant,Mediterranean Restaurant,Greek Restaurant,Fast Food Restaurant,Brazilian Restaurant,New American Restaurant,Sushi Restaurant


In [63]:
# Cluster 2
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[2] + list(range(6, tor_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
15,Christie,Italian Restaurant,Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Doner Restaurant,Comfort Food Restaurant,Gluten-free Restaurant,Colombian Restaurant,Chinese Restaurant


In [64]:
# Cluster 3
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[2] + list(range(6, tor_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,"St. James Town,Cabbagetown",Restaurant,Italian Restaurant,Indian Restaurant,American Restaurant,Thai Restaurant,Taiwanese Restaurant,Caribbean Restaurant,Chinese Restaurant,Japanese Restaurant,Dumpling Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Colombian Restaurant
4,St. James Town,Restaurant,Italian Restaurant,American Restaurant,Seafood Restaurant,Japanese Restaurant,Indian Restaurant,Latin American Restaurant,French Restaurant,German Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,New American Restaurant,Thai Restaurant
5,Berczy Park,Seafood Restaurant,Restaurant,Greek Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Belgian Restaurant,Italian Restaurant,French Restaurant,Comfort Food Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Colombian Restaurant
8,"Harbourfront East,Toronto Islands,Union Station",Italian Restaurant,Restaurant,Indian Restaurant,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Seafood Restaurant,Japanese Restaurant,New American Restaurant,Chinese Restaurant,Doner Restaurant,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Colombian Restaurant
9,"Design Exchange,Toronto Dominion Centre",Restaurant,American Restaurant,Seafood Restaurant,Italian Restaurant,French Restaurant,Asian Restaurant,Fast Food Restaurant,Gluten-free Restaurant,Greek Restaurant,Chinese Restaurant,New American Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Ethiopian Restaurant,Falafel Restaurant
10,"Commerce Court,Victoria Hotel",Restaurant,American Restaurant,Seafood Restaurant,Italian Restaurant,Thai Restaurant,Japanese Restaurant,Latin American Restaurant,French Restaurant,Gluten-free Restaurant,Greek Restaurant,New American Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Ethiopian Restaurant
11,"University of Toronto,Harbord",Restaurant,Japanese Restaurant,Comfort Food Restaurant,Italian Restaurant,Chinese Restaurant,French Restaurant,Sushi Restaurant,Brazilian Restaurant,Caribbean Restaurant,Belgian Restaurant,Colombian Restaurant,German Restaurant,Dim Sum Restaurant,Doner Restaurant,Dumpling Restaurant
13,Stn A PO Boxes 25 The Esplanade,Restaurant,Seafood Restaurant,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,American Restaurant,Thai Restaurant,Belgian Restaurant,Vegetarian / Vegan Restaurant,Molecular Gastronomy Restaurant,French Restaurant,Comfort Food Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Falafel Restaurant


In [65]:
# Cluster 4
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[2] + list(range(6, tor_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
2,"Harbourfront,Regent Park",Mexican Restaurant,French Restaurant,Italian Restaurant,Restaurant,Vietnamese Restaurant,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,Comfort Food Restaurant,Colombian Restaurant


In [66]:
# Cluster 5
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[2] + list(range(6, tor_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
12,"Kensington Market,Grange Park,Chinatown",Vegetarian / Vegan Restaurant,Mexican Restaurant,Dumpling Restaurant,Vietnamese Restaurant,Chinese Restaurant,Caribbean Restaurant,Dim Sum Restaurant,Ramen Restaurant,Comfort Food Restaurant,Doner Restaurant,Belgian Restaurant,Japanese Restaurant,Italian Restaurant,Filipino Restaurant,Brazilian Restaurant
